# Game Score v0.2 Notebook

## Started 11/29/23 to update Game Score Calculation

### Improvments
- adds weight for scoring in close games (team +/- 1 goals)
- Adjusts SHOTS score, utilizes both CLose Shots & Even Strength shots 
- Uses +/- for TEAM score - old methodoligy didn't make any sense
- Adds more depth to PENALTY
    - differenciates and wieghts minor penalties and majors differently
    - GOAL but many too much to chew off right now
        - Uses opposing team's PP success rate to weight the PENALTY score
            - seems easy enough to calculate each teams past PP success but it changes from game to game
                


In [29]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlite3

## Connect to Database
path = '../data/db/Nov_27_YTD_Game_Stats.db'
conn = sqlite3.connect(path)

# Check for tables
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")


# SHOT

In [30]:
# Use the advanced_metrics table to create a dataframe with shots (on_net, missed, blocked) in the EVEN and CLOSE situations

# Load the 'advanced_metrics_combined' table into a DataFrame excluding 'TOTAL' rows
df_amc = pd.read_sql("SELECT * FROM advanced_metrics WHERE player != 'TOTAL'", conn)

# Convert columns to numeric for calculations (replacing empty strings with 0)
columns_to_convert = ['EVEN_Saved', 'EVEN_Goals', 'EVEN_Miss', 'EVEN_Block', 'EVEN_Goals', 
                      'CLOSE_Saved', 'CLOSE_Miss', 'CLOSE_Block', 'CLOSE_Goals', 'CLOSE_Goals',
                        'D_Blocks']
for col in columns_to_convert:
    df_amc[col] = pd.to_numeric(df_amc[col].replace('', '0'))

# Create a new DataFrame with the columns we want to use
shots_df = df_amc[['Team', 'Player', 'Game_ID', 
                   'EVEN_Saved', 'EVEN_Miss', 'EVEN_Block', 'EVEN_Goals',
                   'CLOSE_Saved', 'CLOSE_Miss', 'CLOSE_Block', 'CLOSE_Goals',
                   'D_Blocks']]

# Creat SHOTS ON NET column (Saved + Goals)
shots_df['EVEN_On_Net'] = shots_df['EVEN_Saved'] + shots_df['EVEN_Goals']
shots_df['CLOSE_On_Net'] = shots_df['CLOSE_Saved'] + shots_df['CLOSE_Goals']


# ############ BLOCK TO CALCULATE THE WEIGHTED SHOTS SCORE ############

# SCORE = "CLOSE_On_Net * 0.1 + CLOSE_Miss * 0.08 + CLOSE_Block * 0.05 
#   + EVEN_On_Net * 0.1 + EVEN_Miss * 0.08 + EVEN_Block * 0.05 (NOT D_Blocks - Going to Use in TEAM COMPONANT)"

# Do the calualtions for the weighted shots score and add it to the DataFrame
shots_df['EVEN_Score'] = shots_df['EVEN_On_Net'] * 0.1 + shots_df['EVEN_Miss'] * 0.08 + shots_df['EVEN_Block'] * 0.05
shots_df['CLOSE_Score'] = shots_df['CLOSE_On_Net'] * 0.1 + shots_df['CLOSE_Miss'] * 0.08 + shots_df['CLOSE_Block'] * 0.05

# Add the EVEN and CLOSE scores together to get the total weighted shots score
shots_df['SHOT_Score'] = shots_df['EVEN_Score'] + shots_df['CLOSE_Score']


# shots_df.head()

C:\Users\Justin\AppData\Local\Temp\ipykernel_3064\3030538610.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots_df['EVEN_On_Net'] = shots_df['EVEN_Saved'] + shots_df['EVEN_Goals']
C:\Users\Justin\AppData\Local\Temp\ipykernel_3064\3030538610.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots_df['CLOSE_On_Net'] = shots_df['CLOSE_Saved'] + shots_df['CLOSE_Goals']
C:\Users\Justin\AppData\Local\Temp\ipykernel_3064\3030538610.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of

# FACEOFF

In [31]:
# Calulate Faceoff Score
# SCORE = "Faceoff_Wins * 0.075 - Faceoff_Losses * 0.075"

# Create a new DataFrame with the columns we want to use from player_stats table in the database
df_fo = pd.read_sql("SELECT Team, Player, Game_ID, FOW, FOL FROM player_stats", conn)

# Convert columns to numeric for calculations (replacing empty strings with 0)
columns_to_convert = ['FOW', 'FOL']
for col in columns_to_convert:
    df_fo[col] = pd.to_numeric(df_fo[col].replace('', '0'))

# Do the calualtions for the faceoff score and add it to the DataFrame
df_fo['FACEOFF_Score'] = df_fo['FOW'] * 0.075 - df_fo['FOL'] * 0.075

# Fil Nan values with 0
df_fo = df_fo.fillna(0)

df_fo.head()
# df_fo.info()


,Team,Player,Game_ID,FOW,FOL,FACEOFF_Score
0,Michigan State,Gavin O'Connell,2023-10-07-Lake Superior-Michigan State,0.0,0.0,0.0
1,Michigan State,Tommi Männistö,2023-10-07-Lake Superior-Michigan State,0.0,0.0,0.0
2,Michigan State,Maxim Štrbák,2023-10-07-Lake Superior-Michigan State,0.0,0.0,0.0
3,Michigan State,Artyom Levshunov,2023-10-07-Lake Superior-Michigan State,0.0,0.0,0.0
4,Michigan State,Patrick Geary,2023-10-07-Lake Superior-Michigan State,0.0,0.0,0.0


# TEAM
## +/- *.25 + Blocked Shots * .1

In [32]:
# Calculate TEAM Score 
# Plus-Minus + Defensive Blocks

# Get plus_minus from player_stas table
df_plus_minus = pd.read_sql("SELECT * FROM player_stats WHERE player != 'TOTAL'", conn)
# Simplify the DataFrame to only the columns we need
df_plus_minus = df_plus_minus[['Team', 'Player', 'Game_ID', 'plus_minus']]

# Get Defensive block column from previous dataframe (df_amc)
df_blocks = df_amc[['Team', 'Player', 'Game_ID', 'D_Blocks']]

# Merge the two DataFrames together
df_team = pd.merge(df_plus_minus, df_blocks, how='outer', on=['Team', 'Player', 'Game_ID'])

# Fill any NaN values with 0
df_team = df_team.fillna(0)
# Convert D_Blocks to to int
df_team['D_Blocks'] = df_team['D_Blocks'].astype(int)
# Conver plus_minus to int
df_team['plus_minus'] = df_team['plus_minus'].astype(int)



df_team['TEAM_Score'] = df_team['plus_minus'] * 0.25 + df_team['D_Blocks'] * 0.1

# df_team.sample(25)
# df_team.info()

# df_blocks.info()

# Penalty

In [33]:
# SUB FUNCTION To calculate the the Overall Power Play Success Rate for entire dataset
# - Used to Weight the PENALTY_SCORE

# Count the total number of Power Play (PP) goals from the scoring_summary table.
pp_goals_count = cursor.execute("SELECT COUNT(*) FROM scoring_summary WHERE PP != '';").fetchone()[0]
# Count the total number of Power Plays from the penalty_summary table.
total_pp_count = cursor.execute("SELECT COUNT(*) FROM penalty_summary;").fetchone()[0]
# Calculate the Power Play success rate.
pp_success_rate = pp_goals_count / total_pp_count

## OVERALL NCAA WIDE POWER PLAY SUCCESS RATE TO USE IN FACTOR
print(f'Overall PP % All Games: {pp_success_rate}')


Overall PP % All Games: 0.25161812297734626


In [34]:
# Use Penalty Summary table to get every individual penalty incident
## Count and differentiate between minor and major penalties

# Load the 'penalty_summary' table into a DataFrame
df_pen = pd.read_sql("SELECT * FROM penalty_summary", conn)

# Create Two Columns to count the number of minor and major penalties
df_pen['Minor_Pen'] = np.where(df_pen['Pen_Length'] == '2', 1, 0)
df_pen['Major_Pen'] = np.where(df_pen['Pen_Length'] == '5', 1, 0)

# Group by player, team, and game_id to get the total number of minor and major penalties
df_pen = df_pen.groupby(['Player', 'Team', 'Game_ID']).agg({'Minor_Pen': 'sum', 'Major_Pen': 'sum'}).reset_index()

# Calulate the Penalty Score
# SCORE = "Penalty_Score = Minor_Pen * pp_succes_rate + Major_Pen * pp_success_rate * 2.5"
df_pen['PENALTY_Score'] = df_pen['Minor_Pen'] * pp_success_rate + df_pen['Major_Pen'] * pp_success_rate * 2.5

# df_pen.head(50)
# df_pen.info()

# # How many players have a major penalty?
# df_pen[df_pen['Major_Pen'] > 0].count()

# SCORING

### Method
- use the scoring_summary table to get all scoring incidents
- Identify Scoring Incidents in CLOSE Games - 1 goal game +/-
    - Store as C_Goal, C_Asst1, C_Asst2
- Indentify Scoring that was shorthanded (PP Column will be 'SH')
    - Store as SH_Goal, SH_Asst1, SH_Asst2


In [35]:
# Create SCORING Score using the scoring_summary table

# Load the 'scoring_summary' table into a DataFrame
df_scoring = pd.read_sql("SELECT * FROM scoring_summary", conn)

# Going to Classify Goals and Assists based on game situation so we can weight them differently

# Shorthanded Goals
# df_scoring['SH_Goal'] = np.where(df_scoring['SH'] == 'SH', 1, 0)

# - Identify Scoring Incidents in CLOSE Games - 1 goal game +/-
    # - Store as C_Goal, C_Asst1, C_Asst2

# Out put CSV to TEMP folder for inspection
# df_scoring.to_csv('../TEMP/df_scoring.csv')


In [38]:
# Adjusting the code to correctly classify each goal into one of the three categories
from collections import defaultdict
# Reset the dataframe for reprocessing
df_scoring_updated = df_scoring

# Fill NaN values with empty strings in Assists columns
df_scoring_updated['Assist1'].fillna('NONE', inplace=True)
df_scoring_updated['Assist2'].fillna('NONE', inplace=True)

# Preparing to track scores for each game
game_scores = defaultdict(lambda: {'teams': [], 'scores': {}})
teams_per_game = df_scoring_updated.groupby('Game_ID')['Team'].unique()

# Function to update and check scores for close game situation
def update_and_check_score(row, game_scores):
    # Skip if it's a shorthanded goal
    if row['PP'] == 'SH':
        return False

    game_id = row['Game_ID']
    team = row['Team']

    # Ensure the game has exactly two teams
    if len(game_scores[game_id]['teams']) != 2:
        return False  # Cannot determine close game status without knowing both teams

    # Identify the other team
    other_team = game_scores[game_id]['teams'][0] if team != game_scores[game_id]['teams'][0] else game_scores[game_id]['teams'][1]

    # Update the score for the scoring team
    game_scores[game_id]['scores'][team] += 1

    # Calculate score difference
    score_diff = abs(game_scores[game_id]['scores'][team] - game_scores[game_id]['scores'][other_team])

    # Check if it's a close game situation
    return score_diff <= 1

# Initialize score tracking and process each scoring incident
for game_id, teams in teams_per_game.items():
    if len(teams) == 2:  # Ensure only games with two teams are processed
        game_scores[game_id]['teams'] = teams.tolist()
        game_scores[game_id]['scores'] = {team: 0 for team in teams}

df_scoring_updated['CLOSE'] = df_scoring_updated.apply(lambda row: update_and_check_score(row, game_scores), axis=1)
df_scoring_updated['SHORT'] = df_scoring_updated['PP'] == 'SH'
# Create New Column to identify 'LOW Goals' - If Close_Game = False and SH_Scoring = False
# df_scoring_updated['Low_Goals'] = df_scoring_updated.apply(lambda x: 1 if not x['Close_Game'] and not x['SH_Scoring'] else 0, axis=1)

# Create Bool Column to identify 'Other Goals' - If Close_Game = False and SH_Scoring = False
df_scoring_updated['OTHER'] = df_scoring_updated.apply(lambda x: 1 if not x['CLOSE'] and not x['SHORT'] else 0, axis=1)
df = df_scoring_updated

## Create New DataFrame [Player, Team, Game_ID, Close_Goal, Close_Asst1, Close_Asst2, Short_Goal, Short_Asst1, Short_Asst2, Other_Goal, Other_Asst1, Other_Asst2]

# Create DataFrame for Close Game Goals
df_close_goals = df[df['CLOSE'] == True]
df_close_goals = df_close_goals[['Player', 'Team', 'Game_ID', 'Assist1', 'Assist2']]
# Rename columns to reflect Close Game Goals
df_close_goals = df_close_goals.rename(columns={'Player': 'Close_Goal', 'Assist1': 'Close_Asst1', 'Assist2': 'Close_Asst2'})

# Create DataFrame for Short Handed Goals
df_sh_goals = df[df['SHORT'] == True]
df_sh_goals = df_sh_goals[['Player', 'Team', 'Game_ID', 'Assist1', 'Assist2']]
# Rename columns to reflect Short Handed Goals
df_sh_goals = df_sh_goals.rename(columns={'Assist1': 'Short_Asst1', 'Assist2': 'Short_Asst2'})

# Create DataFrame for Other Goals
df_other_goals = df[df['OTHER'] == 1]
df_other_goals = df_other_goals[['Player', 'Team', 'Game_ID', 'Assist1', 'Assist2']]
# Rename columns to reflect Other Goals
df_other_goals = df_other_goals.rename(columns={'Assist1': 'Other_Asst1', 'Assist2': 'Other_Asst2'})


df_other_goals.head(50)

df = df_other_goals

df.info()

df.sample(25)




<class 'pandas.core.frame.DataFrame'>
Index: 1071 entries, 1 to 2466
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Player       1071 non-null   object
 1   Team         1071 non-null   object
 2   Game_ID      1071 non-null   object
 3   Other_Asst1  1071 non-null   object
 4   Other_Asst2  1071 non-null   object
dtypes: object(5)
memory usage: 50.2+ KB


,Player,Team,Game_ID,Other_Asst1,Other_Asst2
291,Anton Rubtsov,Alaska,2023-10-13-Alaska-Michigan Tech,Arvils Bergmanis,Brady Risk
557,Mac Welsher,Merrimack,2023-10-20-Clarkson-Merrimack,Christian Felton,Mark Hillier
815,Philip Waugh,Mercyhurst,2023-10-26-Mercyhurst-Notre Dame,Spencer Smith,Connor Pelc
249,Jaroslav Chmelar,Providence,2023-10-13-Stonehill-Providence,Liam Valente,Connor Kelley
741,Quinn Olson,Duluth,2023-10-21-Minnesota-Duluth-Bemidji State,NONE,NONE
790,Trevor Griebel,Merrimack,2023-10-22-St. Lawrence-Merrimack,Ty Daneault,Mark Hillier
2416,Massimo Rizzo,Denver,2023-11-25-Yale-Denver,Zeev Buium,McKade Webster
2154,Charles-Alexis Legault,Quinnipiac,2023-11-22-Quinnipiac-Boston University,Mason Marcellus,Andon Cerbone
907,Gavin Brindley,Michigan,2023-10-27-Lindenwood-Michigan,Dylan Duke,T.J. Hughes
2339,Ryan Tattle,Connecticut,2023-11-25-Dartmouth-Connecticut,Joey Muldowney,NONE


In [ ]:
## Pandas Version
pd.__version__

'2.1.1'

In [ ]:
def categorize_assists(row):
    # Initialize categories
    categories = {
        'Assist1_Category': None,
        'Assist2_Category': None
    }

    # If Assist1 is not NaN, categorize based on goal type
    if pd.notna(row['Assist1']):
        if row['SH_Scoring']:
            categories['Assist1_Category'] = 'Short'
        elif row['Close_Game']:
            categories['Assist1_Category'] = 'Close'
        else:
            categories['Assist1_Category'] = 'Other'

    # Repeat for Assist2
    if pd.notna(row['Assist2']):
        if row['SH_Scoring']:
            categories['Assist2_Category'] = 'Short'
        elif row['Close_Game']:
            categories['Assist2_Category'] = 'Close'
        else:
            categories['Assist2_Category'] = 'Other'

    return pd.Series(categories)

# Apply the function to the dataframe
df_scoring_updated[['Assist1_Category', 'Assist2_Category']] = df_scoring_updated.apply(categorize_assists, axis=1)

# df_scoring_updated.head()


KeyError: 'SH_Scoring'

In [ ]:
# Function to count assists in each category
def count_assists(assist_series, category):
    return sum(assist_series == category)

# Aggregating data with the revised categorization
aggregated_data_revised = df_scoring_updated.groupby(['Player', 'Team', 'Game_ID']).agg({
    'SH_Scoring': 'sum',   # Sum of shorthanded goals
    'Close_Game': 'sum',   # Sum of close game goals
    'Other_Goals': 'sum',  # Sum of other goals
    'Assist1_Category': lambda x: count_assists(x, 'Short'),  # Count of shorthanded assists from Assist1
    'Assist2_Category': lambda x: count_assists(x, 'Short'),  # Count of shorthanded assists from Assist2
    'Assist1_Category': lambda x: count_assists(x, 'Close'),  # Count of close game assists from Assist1
    'Assist2_Category': lambda x: count_assists(x, 'Close'),  # Count of close game assists from Assist2
    'Assist1_Category': lambda x: count_assists(x, 'Other'),  # Count of other assists from Assist1
    'Assist2_Category': lambda x: count_assists(x, 'Other')   # Count of other assists from Assist2
}).rename(columns={
    'SH_Scoring': 'Short_Goals',
    'Close_Game': 'Close_Goals',
    'Other_Goals': 'Other_Goals',
    'Assist1_Category': 'Short_Asst1',
    'Assist2_Category': 'Short_Asst2',
    'Assist1_Category': 'Close_Asst1',
    'Assist2_Category': 'Close_Asst2',
    'Assist1_Category': 'Other_Asst1',
    'Assist2_Category': 'Other_Asst2'
}).reset_index()

# Displaying the first few rows of the revised aggregated dataframe
aggregated_data_revised.head()





,Player,Team,Game_ID,Short_Goals,Close_Goals,Other_Goals,Other_Asst1,Other_Asst2
0,A.J. Hodges,Bentley,2023-10-07-Boston University-Bentley,0,1,0,0,0
1,A.J. Hodges,Bentley,2023-10-28-Bentley-Robert Morris,0,0,1,1,1
2,A.J. Hodges,Bentley,2023-11-10-Niagara-Bentley,0,1,2,2,2
3,A.J. Macaulay,Alaska,2023-11-03-Alaska-Anchorage-Alaska,0,0,1,1,1
4,Aaron Bohlinger,Massachusetts,2023-10-13-Michigan-Massachusetts,0,0,1,1,1


In [ ]:
## CHECK GOALS DATAFRAME
# df = goals_df
#Describe the data

df = aggregated_data_revised

# # Creat SUM
# df['TOTAL_G'] = df['Short_Goals'] + df['Close_Goals'] + df['Other_Goals']
# df['TOTAL_A'] = df['Short_Asst1'] + df['Short_Asst2'] + df['Close_Asst1'] + df['Close_Asst2'] + df['Other_Asst1'] + df['Other_Asst2']
# df['TOTAL_P'] = df['TOTAL_G'] + df['TOTAL_A']

                                                                                                                    

df.info()

df.describe()

# Output CSV to TEMP folder for inspection
df.to_csv('../TEMP/df_goals.csv')
# # sort by total goals
# df.sort_values(by=['TOTAL'], ascending=False).head(25)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2203 entries, 0 to 2202
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Player       2203 non-null   object
 1   Team         2203 non-null   object
 2   Game_ID      2203 non-null   object
 3   Short_Goals  2203 non-null   int64 
 4   Close_Goals  2203 non-null   int64 
 5   Other_Goals  2203 non-null   int64 
 6   Other_Asst1  2203 non-null   int64 
 7   Other_Asst2  2203 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 137.8+ KB


In [ ]:
# Extending the categorization logic to include assists (Assist1 and Assist2)

# Function to categorize assists
def categorize_assists(row, assist_col, game_scores):
    # Return None if the assist is NaN
    if pd.isna(row[assist_col]):
        return None

    # Skip if it's related to a shorthanded goal
    if row['PP'] == 'SH':
        return 'SH'

    game_id = row['Game_ID']
    team = row['Team']

    # Ensure the game has exactly two teams
    if len(game_scores[game_id]['teams']) != 2:
        return 'Other'  # Default to 'Other' if conditions are not met

    # Check if it's a close game situation
    if update_and_check_score(row, game_scores):
        return 'Close'
    else:
        return 'Other'

# Categorizing assists
for assist_col in ['Assist1', 'Assist2']:
    df_scoring_updated[f'{assist_col}_Category'] = df_scoring_updated.apply(lambda row: categorize_assists(row, assist_col, game_scores), axis=1)

# Aggregating assist data
aggregated_assists = df_scoring_updated.groupby(['Player', 'Team', 'Game_ID']).agg(
    Short_Assist1=pd.NamedAgg(column='Assist1_Category', aggfunc=lambda x: sum(x == 'SH')),
    Close_Assist1=pd.NamedAgg(column='Assist1_Category', aggfunc=lambda x: sum(x == 'Close')),
    Other_Assist1=pd.NamedAgg(column='Assist1_Category', aggfunc=lambda x: sum(x == 'Other')),
    Short_Assist2=pd.NamedAgg(column='Assist2_Category', aggfunc=lambda x: sum(x == 'SH')),
    Close_Assist2=pd.NamedAgg(column='Assist2_Category', aggfunc=lambda x: sum(x == 'Close')),
    Other_Assist2=pd.NamedAgg(column='Assist2_Category', aggfunc=lambda x: sum(x == 'Other'))
)



aggregated_assists.head()

# # Merging the goals and assists data
# final_aggregated_data = pd.merge(aggregated_data_revised, aggregated_assists, on=['Player', 'Team', 'Game_ID'])

# # Add up Total Points
# final_aggregated_data['Total_Points'] = final_aggregated_data['Short_Goals'] + final_aggregated_data['Close_Goals'] + final_aggregated_data['Other_Goals'] + final_aggregated_data['Short_Assist1'] + final_aggregated_data['Close_Assist1'] + final_aggregated_data['Other_Assist1'] + final_aggregated_data['Short_Assist2'] + final_aggregated_data['Close_Assist2'] + final_aggregated_data['Other_Assist2']


# df = final_aggregated_data 

# # Move TOTAL POINTS in front of SHort_Goals in the DataFrame
# cols = list(df.columns.values) # Make a list of all of the columns in the df
# cols.pop(cols.index('Total_Points')) # Remove TOTAL POINTS from list
# df = df[['Total_Points'] + cols] # Create new dataframe with columns in the order you want

# # df.head()



Short_Assist1  \
Player          Team          Game_ID                                               
A.J. Hodges     Bentley       2023-10-07-Boston University-Bentley              0   
                              2023-10-28-Bentley-Robert Morris                  0   
                              2023-11-10-Niagara-Bentley                        0   
A.J. Macaulay   Alaska        2023-11-03-Alaska-Anchorage-Alaska                0   
Aaron Bohlinger Massachusetts 2023-10-13-Michigan-Massachusetts                 0   

                                                                    Close_Assist1  \
Player          Team          Game_ID                                               
A.J. Hodges     Bentley       2023-10-07-Boston University-Bentley              1   
                              2023-10-28-Bentley-Robert Morris                  0   
                              2023-11-10-Niagara-Bentley                        0   
A.J. Macaulay   Alaska        2023-11-03-Alaska-Anchorage-Alaska                0   
Aaron Bohlinger Massachusetts 2023-10-13-Michigan-Massachusetts                 0   

                                                                    Other_Assist1  \
Player          Team          Game_ID                                               
A.J. Hodges     Bentley       2023-10-07-Boston University-Bentley              0   
                              2023-10-28-Bentley-Robert Morris                  1   
                              2023-11-10-Niagara-Bentley                        3   
A.J. Macaulay   Alaska        2023-11-03-Alaska-Anchorage-Alaska                1   
Aaron Bohlinger Massachusetts 2023-10-13-Michigan-Massachusetts                 1   

                                                                    Short_Assist2  \
Player          Team          Game_ID                                               
A.J. Hodges     Bentley       2023-10-07-Boston University-Bentley              0   
                              2023-10-28-Bentley-Robert Morris                  0   
                              2023-11-10-Niagara-Bentley                        0   
A.J. Macaulay   Alaska        2023-11-03-Alaska-Anchorage-Alaska                0   
Aaron Bohlinger Massachusetts 2023-10-13-Michigan-Massachusetts                 0   

                                                                    Close_Assist2  \
Player          Team          Game_ID                                               
A.J. Hodges     Bentley       2023-10-07-Boston University-Bentley              0   
                              2023-10-28-Bentley-Robert Morris                  0   
                              2023-11-10-Niagara-Bentley                        0   
A.J. Macaulay   Alaska        2023-11-03-Alaska-Anchorage-Alaska                0   
Aaron Bohlinger Massachusetts 2023-10-13-Michigan-Massachusetts                 0   

                                                                    Other_Assist2  
Player          Team          Game_ID                                              
A.J. Hodges     Bentley       2023-10-07-Boston University-Bentley              1  
                              2023-10-28-Bentley-Robert Morris                  1  
                              2023-11-10-Niagara-Bentley                        3  
A.J. Macaulay   Alaska        2023-11-03-Alaska-Anchorage-Alaska                1  
Aaron Bohlinger Massachusetts 2023-10-13-Michigan-Massachusetts                 1

In [ ]:
# Create total Assist column
df = aggregated_assists

df['TOTAL_Assists'] = df['Short_Assist1'] + df['Close_Assist1'] + df['Other_Assist1'] + df['Short_Assist2'] + df['Close_Assist2'] + df['Other_Assist2']

# Sort by TOTAL Assists
df.sort_values(by=['TOTAL_Assists'], ascending=False).head(25)



,,,Short_Assist1,Close_Assist1,Other_Assist1,Short_Assist2,Close_Assist2,Other_Assist2,TOTAL_Assists
Player,Team,Game_ID,,,,,,,
Matt Crasa,Lowell,2023-10-20-Mass.-Lowell-Colgate,0,0,4,0,0,4,8
Jared Westcott,Lake Superior,2023-10-28-Lake Superior-St. Lawrence,0,0,3,0,0,3,6
Dylan Jackson,Arizona State,2023-11-10-Denver-Arizona State,0,0,3,0,0,3,6
Joe Molenaar,St. Cloud State,2023-11-04-Miami-St. Cloud State,0,0,3,0,0,3,6
Isaac Gordon,Michigan Tech,2023-11-03-Augustana-Michigan Tech,0,0,3,0,0,3,6
Daniel Russell,Michigan State,2023-10-19-Canisius-Michigan State,0,0,3,0,0,3,6
Ryan St. Louis,Brown,2023-10-29-Stonehill-Brown,0,0,3,0,0,3,6
Lane Hutson,Boston University,2023-11-04-North Dakota-Boston University,0,2,1,0,0,3,6
Jacques Bouquot,Penn State,2023-11-24-Lindenwood-Penn State,0,0,3,0,0,3,6


In [ ]:
df.describe()

# df.info()

# Sort by TOtal Points
df = df.sort_values(by=['Total_Points'], ascending=False)

df.head(10)

KeyError: 'Total_Points'

In [ ]:
# import pandas as pd
# from collections import defaultdict

# # Function to update and check scores for close game situation
# def update_and_check_score(row, game_scores):
#     game_id = row['Game_ID']
#     team = row['Team']

#     # Ensure the game has exactly two teams
#     if len(game_scores[game_id]['teams']) != 2:
#         return False  # Cannot determine close game status without knowing both teams

#     # Identify the other team
#     other_team = game_scores[game_id]['teams'][0] if team != game_scores[game_id]['teams'][0] else game_scores[game_id]['teams'][1]

#     # Update the score for the scoring team
#     game_scores[game_id]['scores'][team] += 1

#     # Calculate score difference
#     score_diff = abs(game_scores[game_id]['scores'][team] - game_scores[game_id]['scores'][other_team])

#     # Check if it's a close game situation
#     return score_diff <= 1

# # Load the dataframe
# df_scoring_full = df_scoring

# # Prepare to track scores for each game
# teams_per_game = df_scoring_full.groupby('Game_ID')['Team'].unique()
# game_scores = defaultdict(lambda: {'teams': [], 'scores': {}})

# # Initialize score tracking for each game
# for game_id, teams in teams_per_game.items():
#     if len(teams) == 2:  # Ensure only games with two teams are processed
#         game_scores[game_id] = {'teams': teams.tolist(), 'scores': {team: 0 for team in teams}}

# # Process each scoring incident and identify close games vs other  and shorthanded goals
# df_scoring_full['Close_Game'] = df_scoring_full.apply(lambda row: update_and_check_score(row, game_scores), axis=1)

# # Tag "Other" goals
# # df_scoring_full['Other'] = df_scoring_full['Close_Game'] == False

# # Tag Shorthanded Goals
# df_scoring_full['SH_Scoring'] = df_scoring_full['PP'] == 'SH'




In [ ]:
# Calculating 'Other_Goals'
df_scoring_full['Other_Goals'] = df_scoring_full.apply(lambda x: 1 if not x['Close_Game'] and not x['SH_Scoring'] else 0, axis=1)
# Other_Assist1 & Other_Assist2
df_scoring_full['Other_Assist1'] = df_scoring_full.apply(lambda x: 1 if pd.notna(x['Assist1']) and not x['Close_Game'] and not x['SH_Scoring'] else 0, axis=1)
df_scoring_full['Other_Assist2'] = df_scoring_full.apply(lambda x: 1 if pd.notna(x['Assist2']) and not x['Close_Game'] and not x['SH_Scoring'] else 0, axis=1)


# Aggregating data
aggregated_data = df_scoring_full.groupby(['Player', 'Team', 'Game_ID']).agg(
    Short_Goals=pd.NamedAgg(column='SH_Scoring', aggfunc='sum'),
    Close_Goals=pd.NamedAgg(column='Close_Game', aggfunc='sum'),
    Other_Goals=pd.NamedAgg(column='Other_Goals', aggfunc='sum'),
    # Short_Assist1=pd.NamedAgg(column='SH_Scoring', aggfunc=lambda x: x.shift(-1).sum()),
)



# Adding assist columns
for assist_col in ['Assist1', 'Assist2']:
    aggregated_data[f'Short_{assist_col}'] = df_scoring_full[df_scoring_full['SH_Scoring']].groupby(['Player', 'Team', 'Game_ID'])[assist_col].count()
    aggregated_data[f'Close_{assist_col}'] = df_scoring_full[df_scoring_full['Close_Game']].groupby(['Player', 'Team', 'Game_ID'])[assist_col].count()
    aggregated_data[f'Other_{assist_col}'] = df_scoring_full[df_scoring_full['Other_Goals']].groupby(['Player', 'Team', 'Game_ID'])[assist_col].count()

# # Calculating 'Other_Assists'
# for assist_col in ['Assist1', 'Assist2']:
#     df_scoring_full['Other_Assist'] = df_scoring_full.apply(lambda x: 1 if pd.notna(x[assist_col]) and not x['Close_Game'] and not x['SH_Scoring'] else 0, axis=1)


# Resetting index
aggregated_data.reset_index(inplace=True)

# Displaying the first few rows of the aggregated dataframe
aggregated_data.head()



In [ ]:


score_df = aggregated_data

score_df.describe()

In [ ]:
### Calculate SCORING Score

# score_df = aggregated_data

# Fill Nan values with 0
score_df = score_df.fillna(0)

non_int_cols = ['Player', 'Team', 'Game_ID']

# Convert all columns to int except for non_int_cols
for col in score_df.columns:
    if col not in non_int_cols:
        score_df[col] = score_df[col].astype(int)

# Calculate the Scoring Score
## Factors - GOALS = .555, ASSIST1 = .455, ASSIST2 = .355
## CLOSE GAME MULTIPLIER = 1.5
## SH GOAL MULTIPLIER = 1.75

score_df['SCORING_Score'] = score_df['Close_Goals'] * 1.5 * 0.555 + score_df['Close_Assist1'] * 1.5 * 0.455 + score_df['Close_Assist2'] * 1.5 * 0.355 + \
                            score_df['Short_Goals'] * 1.75 * 0.555 + score_df['Short_Assist1'] * 1.75 * 0.455 + score_df['Short_Assist2'] * 1.75 * 0.355 + \
                            score_df['Other_Goals'] * 0.555 + score_df['Other_Assist1'] * 0.455 + score_df['Other_Assist2'] * 0.355

score_df.head()



In [ ]:
## Examine Output 

# Histogram of Scoring Score
score_df['SCORING_Score'].hist()


score_df.info()

# Sort by Scoring Score and show top 25
score_df.sort_values(by='SCORING_Score', ascending=False).head(25)

# Value Counts for Short_Assist2
# score_df['Short_Assist2'].value_counts()

## Combine to single dataframe

In [ ]:
### COMBINE ALL THE DATAFRAMES TOGETHER

# Merge the shots_df and df_fo DataFrames together
df_combined = pd.merge(shots_df, df_fo, how='outer', on=['Team', 'Player', 'Game_ID'])

# Merge the df_combined and df_team DataFrames together
df_combined = pd.merge(df_combined, df_team, how='outer', on=['Team', 'Player', 'Game_ID'])

# Merge the df_combined and df_pen DataFrames together
df_combined = pd.merge(df_combined, df_pen, how='outer', on=['Team', 'Player', 'Game_ID'])

# Fill any NaN values with 0
df_combined = df_combined.fillna(0)

# Filter To Just The SCORE Columns
keep_col = ['Team', 'Player', 'Game_ID', 'SHOT_Score', 'FACEOFF_Score', 'TEAM_Score', 'PENALTY_Score']
df_combined = df_combined[keep_col]

# Calulate the TOTAL Score
## SCORE = "TOTAL_Score = SHOT_Score + FACEOFF_Score + TEAM_Score - PENALTY_Score"
df_combined['TOTAL_Score'] = df_combined['SHOT_Score'] + df_combined['FACEOFF_Score'] + df_combined['TEAM_Score'] - df_combined['PENALTY_Score']

df_combined.head(10)

df_combined.info()

In [ ]:
# Score Distribution
df_combined['TOTAL_Score'].describe()

 # Histogram Plot of Score Distribution
df_combined['TOTAL_Score'].hist(bins=30)

# Histograms of all the scores
df_combined.hist(bins=30, figsize=(20,15))